In [20]:
file_date = '2021-04-18'

<div style="max-width:1400px;margin-center: auto">
<img src="images\drivers.png" width="600"/>
</div>

In [21]:
import findspark
findspark.init()
import pyspark
from delta import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

#  Create a spark session with Delta
builder = pyspark.sql.SparkSession.builder.appName("dbcreation") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Create spark context
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [22]:
from pyspark.sql.types import StringType, DateType, IntegerType, StructType, StructField
from pyspark.sql.functions import current_timestamp, concat, lit, col

In [23]:
name_schema = StructType(fields = [StructField("forename", StringType(), True),
                                    StructField("surname", StringType(), True)])

In [24]:
drivers_schema = StructType(fields = [StructField("driverId", IntegerType(), False),
                                      StructField("driverRef", StringType(), False),
                                      StructField("number", IntegerType(), False),
                                      StructField("code", StringType(), False),
                                      StructField("name", name_schema),
                                      StructField("dob", DateType(), False),
                                      StructField("nationality", StringType(), False),
                                      StructField("url", StringType(), False)])

In [25]:
drivers_df = spark.read.json(f'raw files\\{file_date}\\drivers.json',schema=drivers_schema)
drivers_df.show(2)

+--------+---------+------+----+-----------------+----------+-----------+--------------------+
|driverId|driverRef|number|code|             name|       dob|nationality|                 url|
+--------+---------+------+----+-----------------+----------+-----------+--------------------+
|       1| hamilton|    44| HAM|{Lewis, Hamilton}|1985-01-07|    British|http://en.wikiped...|
|       2| heidfeld|  null| HEI| {Nick, Heidfeld}|1977-05-10|     German|http://en.wikiped...|
+--------+---------+------+----+-----------------+----------+-----------+--------------------+
only showing top 2 rows



In [26]:
drivers_trans_df = drivers_df.withColumnRenamed("driverId","driver_id").\
                             withColumnRenamed("driverRef","driver_ref").\
                             withColumn("Ingestion_date",current_timestamp()).\
                             withColumn("name",concat(col("name.forename"),lit(" "),col("name.surname"))).\
                            withColumn('file_date',lit(file_date))

In [27]:
drivers_fnl_df = drivers_trans_df.drop("url")

In [28]:
drivers_fnl_df.write.mode('overwrite').format('delta').saveAsTable('default.drivers')